In [2]:
# %pip install pyspark==3.4.1

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from pyspark.sql.functions import array, col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [4]:
import os
cwd = os.getcwd()

In [5]:
# spark = SparkSession.builder.master("local[1]") \
#                     .appName('ItemCodes') \
#                     .getOrCreate()
spark = SparkSession.builder \
                    .master("spark://127.0.0.1:7077") \
                    .appName('ItemCodes') \
                    .getOrCreate()

In [6]:
pcl_df = spark.read.option("header", True)\
    .csv(f"{cwd}/dataset/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_ItemCodes.csv")
pcl_df.printSchema()

root
 |-- Item Code: string (nullable = true)
 |-- CPC Code: string (nullable = true)
 |-- Item: string (nullable = true)



In [7]:
pcl_df = pcl_df.select(
    pcl_df['Item Code'].alias('id'),
    pcl_df['CPC Code'].alias('item_code_cpc'),
    pcl_df['Item'].alias('item')
)
pcl_df.show()

+----+-------------+--------------------+
|  id|item_code_cpc|                item|
+----+-------------+--------------------+
| 101|       '01195|         Canary seed|
|1016|       '02123|               Goats|
|1017|       '21116|Meat of goat; fre...|
|1018|       '21156|Edible offal of g...|
|1019|       '21515|Goat fat; unrendered|
|1020|       '02292|   Raw milk of goats|
|1021|       '22254|Cheese from milk ...|
|1022|    '22249.02| Butter of goat milk|
|1025|       '02954|Raw hides and ski...|
| 103|    '01199.02|         Mixed grain|
|1034|       '02140|        Swine / pigs|
|1035|    '21113.01|Meat of pig with ...|
|1036|       '21153|Edible offal of p...|
|1037|    '21511.01|         Fat of pigs|
|1043|       '21521|   Pig fat; rendered|
|1057|       '02151|            Chickens|
|1058|       '21121|Meat of chickens;...|
|1062|        '0231|Hen eggs in shell...|
|1068|       '02154|               Ducks|
|1069|       '21122|Meat of ducks; fr...|
+----+-------------+--------------

In [8]:
pcl_df = pcl_df.orderBy('item_code_cpc')
pcl_df.show()

+---+-------------+--------------------+
| id|item_code_cpc|                item|
+---+-------------+--------------------+
| 15|        '0111|               Wheat|
| 56|        '0112|        Maize (corn)|
| 27|        '0113|                Rice|
| 83|        '0114|             Sorghum|
| 44|        '0115|              Barley|
| 71|        '0116|                 Rye|
| 75|        '0117|                Oats|
| 79|        '0118|              Millet|
| 97|       '01191|           Triticale|
| 89|       '01192|           Buckwheat|
| 94|       '01193|               Fonio|
| 92|       '01194|              Quinoa|
|101|       '01195|         Canary seed|
|103|    '01199.02|         Mixed grain|
|108|    '01199.90|      Cereals n.e.c.|
|367|       '01211|           Asparagus|
|358|       '01212|            Cabbages|
|393|       '01213|Cauliflowers and ...|
|372|       '01214| Lettuce and chicory|
|373|       '01215|             Spinach|
+---+-------------+--------------------+
only showing top

In [9]:
CROPS_PRIMARY = ['Abaca; manila hemp; raw', 'Agave fibres; raw; n.e.c.', 'Almonds; in shell', 'Anise; badian; coriander; cumin; caraway; fennel and juniper berries; raw', 'Apples', 'Apricots', 'Areca nuts', 'Artichokes', 'Asparagus', 'Avocados', 'Balata; gutta-percha; guayule; chicle and similar natural gums in primary forms or in plates; sheets or strip', 'Bambara beans; dry', 'Bananas', 'Barley', 'Beans; dry', 'Blueberries', 'Brazil nuts; in shell', 'Broad beans and horse beans; dry', 'Broad beans and horse beans; green', 'Buckwheat', 'Cabbages', 'Canary seed', 'Cantaloupes and other melons', 'Carrots and turnips', 'Cashew nuts; in shell', 'Cashewapple', 'Cassava leaves', 'Cassava; fresh', 'Castor oil seeds', 'Cauliflowers and broccoli', 'Cereals n.e.c.', 'Cherries', 'Chestnuts; in shell', 'Chick peas; dry', 'Chicory roots', 'Chillies and peppers; dry (Capsicum spp.; Pimenta spp.); raw', 'Chillies and peppers; green (Capsicum spp. and Pimenta spp.)', 'Cinnamon and cinnamon-tree flowers; raw', 'Cloves (whole stems); raw', 'Cocoa beans', 'Coconuts; in shell', 'Coffee; green', 'Coir; raw', 'Cow peas; dry', 'Cranberries', 'Crops; primary', 'Cucumbers and gherkins', 'Currants', 'Dates', 'Edible roots and tubers with high starch or inulin content; n.e.c.; fresh', 'Eggplants (aubergines)', 'Figs', 'Flax; processed but not spun', 'Fonio', 'Ginger; raw', 'Gooseberries', 'Grapes', 'Green corn (maize)', 'Green garlic', 'Groundnuts; excluding shelled', 'Hazelnuts; in shell', 'Hempseed', 'Hop cones', 'Jojoba seeds', 'Jute; raw or retted', 'Kapok fruit', 'Karite nuts (sheanuts)', 'Kenaf; and other textile bast fibres; raw or retted', 'Kiwi fruit', 'Kola nuts', 'Leeks and other alliaceous vegetables', 'Lemons and limes', 'Lentils; dry', 'Lettuce and chicory', 'Linseed', 'Locust beans (carobs)', 'Lupins', 'Maize (corn)', 'Mangoes; guavas and mangosteens', 'Maté leaves', 'Melonseed', 'Millet', 'Mixed grain', 'Mushrooms and truffles', 'Mustard seed', 'Natural rubber in primary forms', 'Nutmeg; mace; cardamoms; raw', 'Oats', 'Oil palm fruit', 'Okra', 'Olives', 'Onions and shallots; dry (excluding dehydrated)', 'Onions and shallots; green', 'Oranges', 'Other beans; green', 'Other berries and fruits of the genus vaccinium n.e.c.', 'Other citrus fruit; n.e.c.', 'Other fibre crops; raw; n.e.c.', 'Other fruits; n.e.c.', 'Other nuts (excluding wild edible nuts and groundnuts); in shell; n.e.c.', 'Other oil seeds; n.e.c.', 'Other pome fruits', 'Other pulses n.e.c.', 'Other stimulant; spice and aromatic crops; n.e.c.', 'Other stone fruits', 'Other sugar crops n.e.c.', 'Other tropical fruits; n.e.c.', 'Other vegetables; fresh n.e.c.', 'Papayas', 'Peaches and nectarines', 'Pears', 'Peas; dry', 'Peas; green', 'Pepper (Piper spp.); raw', 'Peppermint; spearmint', 'Persimmons', 'Pigeon peas; dry', 'Pineapples', 'Pistachios; in shell', 'Plantains and cooking bananas', 'Plums and sloes', 'Pomelos and grapefruits', 'Poppy seed', 'Potatoes', 'Pumpkins; squash and gourds', 'Pyrethrum; dried flowers', 'Quinces', 'Quinoa', 'Ramie; raw or retted', 'Rape or colza seed', 'Raspberries', 'Rice', 'Rye', 'Safflower seed', 'Seed cotton; unginned', 'Sesame seed', 'Sisal; raw', 'Sorghum', 'Sour cherries', 'Soya beans', 'Spinach', 'Strawberries', 'String beans', 'Sugar beet', 'Sugar cane', 'Sunflower seed', 'Sweet potatoes', 'Tallowtree seeds', 'Tangerines; mandarins; clementines', 'Taro', 'Tea leaves', 'Tomatoes', 'Triticale', 'True hemp; raw or retted', 'Tung nuts', 'Unmanufactured tobacco', 'Vanilla; raw', 'Vetches', 'Walnuts; in shell', 'Watermelons', 'Wheat', 'Yams', 'Yautia']
CROPS_PROCESSED = ['Beer of barley; malted', 'Coconut oil', 'Cotton lint; ginned', 'Cotton seed', 'Cottonseed oil', 'Crops Processed', 'Green tea (not fermented); black tea (fermented) and partly fermented tea', 'Groundnut oil', 'Kapok fibre; raw', 'Kapokseed in shell', 'Margarine and shortening', 'Molasses', 'Oil of linseed', 'Oil of maize', 'Oil of palm kernel', 'Oil of sesame seed', 'Olive oil', 'Palm kernels', 'Palm oil', 'Rapeseed or canola oil; crude', 'Raw cane or beet sugar (centrifugal only)', 'Safflower-seed oil; crude', 'Soya bean oil', 'Sunflower-seed oil; crude', 'Wine']
LIVESTOCK_PRIMARY = ['Beeswax', 'Buffalo fat; unrendered', 'Cattle fat; unrendered', 'Edible offal of buffalo; fresh; chilled or frozen', 'Edible offal of cattle; fresh; chilled or frozen', 'Edible offal of goat; fresh; chilled or frozen', 'Edible offal of pigs; fresh; chilled or frozen', 'Edible offal of sheep; fresh; chilled or frozen', 'Edible offals of camels and other camelids; fresh; chilled or frozen', 'Edible offals of horses and other equines;  fresh; chilled or frozen', 'Eggs from other birds in shell; fresh; n.e.c.', 'Fat of camels', 'Fat of pigs', 'Game meat; fresh; chilled or frozen', 'Goat fat; unrendered', 'Hen eggs in shell; fresh', 'Horse meat; fresh or chilled', 'Livestock primary', 'Meat of asses; fresh or chilled', 'Meat of buffalo; fresh or chilled', 'Meat of camels; fresh or chilled', 'Meat of cattle with the bone; fresh or chilled', 'Meat of chickens; fresh or chilled', 'Meat of ducks; fresh or chilled', 'Meat of geese; fresh or chilled', 'Meat of goat; fresh or chilled', 'Meat of mules; fresh or chilled', 'Meat of other domestic camelids; fresh or chilled', 'Meat of other domestic rodents; fresh or chilled', 'Meat of pig with the bone; fresh or chilled', 'Meat of pigeons and other birds n.e.c.; fresh; chilled or frozen', 'Meat of rabbits and hares; fresh or chilled', 'Meat of sheep; fresh or chilled', 'Meat of turkeys; fresh or chilled', 'Natural honey', 'Other meat n.e.c. (excluding mammals); fresh; chilled or frozen', 'Raw hides and skins of buffaloes', 'Raw hides and skins of cattle', 'Raw hides and skins of goats or kids', 'Raw hides and skins of sheep or lambs', 'Raw milk of buffalo', 'Raw milk of camel', 'Raw milk of cattle', 'Raw milk of goats', 'Raw milk of sheep', 'Sheep fat; unrendered', 'Shorn wool; greasy; including fleece-washed shorn wool', 'Silk-worm cocoons suitable for reeling', 'Snails; fresh; chilled; frozen; dried; salted or in brine; except sea snails']
LIVESTOCK_PROCESSED = ['Butter and ghee of sheep milk', 'Butter of buffalo milk', 'Butter of cow milk', 'Butter of goat milk', 'Buttermilk; dry', 'Cheese from milk of buffalo; fresh or processed', 'Cheese from milk of goats; fresh or processed', 'Cheese from milk of sheep; fresh or processed', 'Cheese from skimmed cow milk', 'Cheese from whole cow milk', 'Cream; fresh', 'Ghee from buffalo milk', 'Ghee from cow milk', 'Livestock processed', 'Pig fat; rendered', 'Raw silk (not thrown)', 'Skim milk and whey powder', 'Skim milk of cows', 'Skim milk; condensed', 'Skim milk; evaporated', 'Tallow', 'Whey; condensed', 'Whey; dry', 'Whole milk powder', 'Whole milk; condensed', 'Whole milk; evaporated', 'Yoghurt']
LIVE_ANIMAL = ['Asses', 'Bees', 'Buffalo', 'Camels', 'Cattle', 'Chickens', 'Ducks', 'Geese', 'Goats', 'Horses', 'Live Animals', 'Mules and hinnies', 'Other birds', 'Other camelids', 'Other rodents', 'Rabbits and hares', 'Sheep', 'Swine / pigs', 'Turkeys']

# CATEGORY = {}
# def list_to_dict(lis, name):
#     for x in lis:
#         CATEGORY[x] = name

# list_to_dict(CROPS_PRIMARY, 'CROPS_PRIMARY')
# list_to_dict(CROPS_PROCESSED, 'CROPS_PROCESSED')
# list_to_dict(LIVESTOCK_PRIMARY, 'LIVESTOCK_PRIMARY')
# list_to_dict(LIVESTOCK_PROCESSED, 'LIVESTOCK_PROCESSED')
# list_to_dict(LIVE_ANIMAL, 'LIVE_ANIMAL')

In [10]:
pcl_df = pcl_df.withColumn("item_category", F.when(F.col('item').isin(CROPS_PRIMARY), F.lit('CROPS_PRIMARY'))
                                            .when(F.col('item').isin(CROPS_PROCESSED), F.lit('CROPS_PROCESSED'))
                                            .when(F.col('item').isin(LIVESTOCK_PRIMARY), F.lit('LIVESTOCK_PRIMARY'))
                                            .when(F.col('item').isin(LIVESTOCK_PROCESSED), F.lit('LIVESTOCK_PROCESSED'))
                                            .when(F.col('item').isin(LIVE_ANIMAL), F.lit('LIVE_ANIMAL'))
                                            .otherwise(None))
pcl_df.show()

+---+-------------+--------------------+-------------+
| id|item_code_cpc|                item|item_category|
+---+-------------+--------------------+-------------+
| 15|        '0111|               Wheat|CROPS_PRIMARY|
| 56|        '0112|        Maize (corn)|CROPS_PRIMARY|
| 27|        '0113|                Rice|CROPS_PRIMARY|
| 83|        '0114|             Sorghum|CROPS_PRIMARY|
| 44|        '0115|              Barley|CROPS_PRIMARY|
| 71|        '0116|                 Rye|CROPS_PRIMARY|
| 75|        '0117|                Oats|CROPS_PRIMARY|
| 79|        '0118|              Millet|CROPS_PRIMARY|
| 97|       '01191|           Triticale|CROPS_PRIMARY|
| 89|       '01192|           Buckwheat|CROPS_PRIMARY|
| 94|       '01193|               Fonio|CROPS_PRIMARY|
| 92|       '01194|              Quinoa|CROPS_PRIMARY|
|101|       '01195|         Canary seed|CROPS_PRIMARY|
|103|    '01199.02|         Mixed grain|CROPS_PRIMARY|
|108|    '01199.90|      Cereals n.e.c.|CROPS_PRIMARY|
|367|     

In [11]:
pcl_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Item_Prod")

In [12]:
spark.stop()